In [1]:
# import libraries

import os
import pandas as pd
from datetime import date
import pickle

## Using sunscrape to supplement denier tweets ##
- Since 2019-06-01: 'climatechangeisfalse', 'climatechangenotreal', 'climatechangehoax', 'globalwarminghoax' (around 30,000, before hydrating), climatedeniers (6,000), climatehoax (68,000)
- Since 2017-09-21 (missing hashtags): climatechangescam(1484), climatehysteria(11,000), globalwarmingisahoax (774), globalwarmingscam(600), globalcooling(10,000)
- https://firstdraftnews.org/articles/climate-change-misinformation-conspiracy-memes/
- https://awario.com/blog/climate-change-denial/


In [5]:
today = date.today()
end_date = today

In [45]:
search_term = 'globalcooling'
from_date = '2017-09-21'

In [46]:
os.system(f"snscrape --since {from_date} twitter-search '{search_term} until:{end_date}' > result-tweets.txt")

if os.stat("result-tweets.txt").st_size == 0:
    counter = 0
else:
    df = pd.read_csv('result-tweets.txt', names=['link'])
    counter = df.size

print('Number Of Tweets : '+ str(counter))

Number Of Tweets : 10306


In [8]:
climate_hoax = pd.read_csv('result-tweets.txt')

In [12]:
climate_hoax.to_csv('climate_hoax.csv')

In [48]:
filenames = ['climate_false.txt', 'climate_not_real.txt', 'climatechangehoax.txt',
            'climatechangescam.txt', 'climatedeniers.txt', 'climatehoax.txt',
            'climatehysteria.txt', 'globalcooling.txt','globalwarminghoax.txt',
            'globalwarmingisahoax.txt', 'globalwarmingscam.txt']

with open('denier_supplements.txt', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

In [61]:
data = pd.read_csv('denier_supplements.txt', header=None)
data

,0
0,https://twitter.com/RClarkofNC/status/13315671...
1,https://twitter.com/baeareavibez/status/132707...
2,https://twitter.com/blessedpolish/status/13065...
3,https://twitter.com/Kringdom1/status/117585894...
4,https://twitter.com/20Fireblade13/status/11581...
...,...
127799,https://twitter.com/TeamTrumpUSA/status/918519...
127800,https://twitter.com/100monkeys/status/91811357...
127801,https://twitter.com/JohnInLongmont/status/9174...
127802,https://twitter.com/howardbrown7/status/917017...


In [62]:
data1 = data
data1.columns = ['url']
data1

,url
0,https://twitter.com/RClarkofNC/status/13315671...
1,https://twitter.com/baeareavibez/status/132707...
2,https://twitter.com/blessedpolish/status/13065...
3,https://twitter.com/Kringdom1/status/117585894...
4,https://twitter.com/20Fireblade13/status/11581...
...,...
127799,https://twitter.com/TeamTrumpUSA/status/918519...
127800,https://twitter.com/100monkeys/status/91811357...
127801,https://twitter.com/JohnInLongmont/status/9174...
127802,https://twitter.com/howardbrown7/status/917017...


In [ ]:
.iloc[:,[0,1]]

In [65]:
separated = data1['url'].str.split('/', expand = True)

In [67]:
separated.columns = ['one', 'two','three','four','five','six']

In [71]:
ids = separated['six']

In [74]:
ids = ids.drop_duplicates()

In [75]:
ids

0         1331567111735533569
1         1327071840209051648
2         1306503581709807616
3         1175858943668740098
4         1158157717439209475
                 ...         
127799     918519356253986817
127800     918113571095089153
127801     917409761494274048
127802     917017030330671104
127803     914918334080020480
Name: six, Length: 124753, dtype: object

In [76]:
ids.to_csv('denier_ids.txt', header=False, index=False, sep='\t')

## Building training and prediction datasets

In [2]:
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)

In [4]:
# Import 2017-2019 dataset with climate believer/denier hashtags

hashtags = pd.read_csv('../datasets/tweets_hashtags.csv', low_memory=False)
print('hashtags shape:',hashtags.shape)

# Import 2017-2019 dataset with geotags

geotags = pd.read_csv('../datasets/climate_tweet_ids.csv',low_memory=False)
print('geotags shape:',geotags.shape)

# Import denier supplement dataset

denier_supp = pd.read_csv('../datasets/denier_supp.csv',low_memory=False)
print('denier_supp shape:',denier_supp.shape)

hashtags shape: (587817, 35)
geotags shape: (321316, 35)
denier_supp shape: (124740, 35)


In [5]:
dfs = [hashtags, geotags, denier_supp]

if all([set(dfs[0].columns) == set(df.columns) for df in dfs]):
    print('All have the same columns')
else:
    print('Some have different columns')

All have the same columns


In [6]:
# Concatenate the three datasets

dfs = [hashtags, geotags, denier_supp]
df = pd.concat(dfs, axis = 0).drop_duplicates().reset_index(drop=True)
df.shape

(1033560, 35)

In [7]:
cols = [0,1,3,4,7,8,9,12,13,15,16,17]
df.drop(df.columns[cols], axis=1, inplace=True)


df = df.drop(df.loc[:,'tweet_url':'user_listed_count'].columns, axis = 1)
df = df.drop(df.loc[:, 'user_name':'user_verified'].columns, axis = 1)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1033560 entries, 0 to 1033559
Data columns (total 8 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   hashtags        801703 non-null   object
 1   favorite_count  1033560 non-null  int64 
 2   id              1033560 non-null  int64 
 3   lang            1033560 non-null  object
 4   place           332208 non-null   object
 5   retweet_count   1033560 non-null  int64 
 6   text            1033560 non-null  object
 7   user_location   896589 non-null   object
dtypes: int64(3), object(5)
memory usage: 63.1+ MB


In [10]:
df.head()

,hashtags,favorite_count,id,lang,place,retweet_count,text,user_location
0,climatechange climateaction sustainability fb,0,930123883520413697,en,NaN,0,We are not going to get too many more warnings...,"Holmdel, NJ"
1,juice reuse reduce actonclimate,1,965587085993365505,en,NaN,1,I'll be juicing my left over carrots tonight t...,"Miami, FL"
2,Hack4Climate Blockchain ClimateAction ClimateC...,4,928605716159574016,en,NaN,1,In less than 3 days we will #Hack4Climate at @...,"Zürich, Schweiz"
3,climatechange climateaction,1,954288658361802752,en,NaN,1,"""The best way to predict the future is to crea...","Tampere, Finland"
4,protectwhatyoulove climatechangeisreal AnimalR...,0,1012691887440814086,en,NaN,0,Accepting the world for the way it is just mig...,NaN


In [11]:
# Drop non-English tweets

indexes = df[ df['lang'] != 'en' ].index
df.drop(indexes, inplace = True)
  
df.shape

(927487, 8)

In [12]:
# save full dataset

df.to_pickle("../data/full_data.pkl")

In [72]:
df['user_location'].value_counts()

Globally l Planet Earth          72662
Tampere, Finland                 58831
Globally l Online l Earth        39759
Online l Globally l Earth        39524
United States                    13881
                                 ...  
Bondi Beach and virtual space        1
West Orlando,Florida                 1
Earth !                              1
central ohio                         1
Infidellia                           1
Name: user_location, Length: 62872, dtype: int64

In [73]:
# Process 'user_location' variable to separate it into city and state

data1 = df.dropna(how='any', subset=['user_location'])
data1 = data1[data1["user_location"].str.contains(",")]
location = data1['user_location'].str.split(',', expand = True).iloc[:,[0,1]]
location.columns = ['city','state']
location

,city,state
0,Holmdel,NJ
1,Miami,FL
2,Zürich,Schweiz
3,Tampere,Finland
11,Cairns,Queensland
...,...,...
1033548,Overland Park,KS
1033551,Longmont,CO
1033552,Texas,U.S.A.
1033554,Longmont,CO


In [74]:
# Add the two new columns back into the dataframe

data1 = pd.concat([data1, location],axis = 1, join = 'outer', 
                 ignore_index=False, sort=False)

# Strip the new columns of any spaces

data1.state = data1.state.str.strip()
data1.city = data1.city.str.strip()

In [75]:
# Filter dataframe to only rows that have a US state abbreviation

states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY", 
          "Alabama","Alaska","Arizona","Arkansas","California","Colorado",
          "Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois",
          "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
          "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
          "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
          "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
          "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
          "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]

data1 = data1[data1['state'].isin(states)]

data1.shape

(133082, 10)

In [76]:
data1[['user_location', 'city', 'state']]

,user_location,city,state
0,"Holmdel, NJ",Holmdel,NJ
1,"Miami, FL",Miami,FL
13,"Monterey County, CA, USA",Monterey County,CA
29,"Spokane Valley, WA",Spokane Valley,WA
45,"Miami, FL",Miami,FL
...,...,...,...
1033537,"Longmont, CO",Longmont,CO
1033544,"Longmont, CO",Longmont,CO
1033548,"Overland Park, KS",Overland Park,KS
1033551,"Longmont, CO",Longmont,CO


In [77]:
# Process 'place' variable to separate it into city and state

data2 = df.dropna(how='any', subset=['place'])
data2 = data2[data2["place"].str.contains(",")]
place = data2['place'].str.split(',', expand = True).iloc[:,[0,1]]
place.columns = ['city','state']
place

,city,state
112,Morogoro,Tanzania
179,Twentynine Palms,CA
231,Aiton,France
241,Salt Lake City,UT
311,New Orleans,LA
...,...,...
1033267,Lehigh Acres,FL
1033308,Winnipeg,Manitoba
1033359,Charlotte,NC
1033510,Los Angeles,CA


In [78]:
# Add the two new columns back into the dataframe

data2 = pd.concat([data2, place],axis = 1, join = 'outer', 
                 ignore_index=False, sort=False)

# Strip the new columns of any spaces

data2.state = data2.state.str.strip()
data2.city = data2.city.str.strip()

In [79]:
# Filter dataframe to only rows that have a US state abbreviation

data2 = data2[data2['state'].isin(states)]

data2.shape

(129754, 10)

In [81]:
# Concatenate the three datasets

datas = [data1, data2]
data_final = pd.concat(datas).drop_duplicates().reset_index(drop=True)
data_final.shape

(229795, 10)

In [82]:
# USA State abbreviations to make everything in state variable 2-letter

us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

# invert the dictionary
# abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))

In [83]:
#Map a dictionary of abbreviations to the dataframe
data_final['state'] = data_final['state'].map(us_state_to_abbrev).fillna(data_final['state'])
data_final.shape

(229795, 10)

In [84]:
data_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229795 entries, 0 to 229794
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   hashtags        112176 non-null  object
 1   favorite_count  229795 non-null  int64 
 2   id              229795 non-null  int64 
 3   lang            229795 non-null  object
 4   place           165768 non-null  object
 5   retweet_count   229795 non-null  int64 
 6   text            229795 non-null  object
 7   user_location   214065 non-null  object
 8   city            229795 non-null  object
 9   state           229795 non-null  object
dtypes: int64(3), object(7)
memory usage: 17.5+ MB


In [85]:
data_final.to_pickle("usa_tweets.pkl")

## Creating Training/Test Dataset

In [86]:
# Drop NA's in hashtags column

df = df.dropna(how='any', subset=['hashtags'])
df.shape

(716046, 8)

In [87]:
# Drop any tweets that appear in USA tweets dataframe

condition = df['id'].isin(data_final['id'])
df.drop(df[condition].index, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 607346 entries, 2 to 1033559
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   hashtags        607346 non-null  object
 1   favorite_count  607346 non-null  int64 
 2   id              607346 non-null  int64 
 3   lang            607346 non-null  object
 4   place           69763 non-null   object
 5   retweet_count   607346 non-null  int64 
 6   text            607346 non-null  object
 7   user_location   529331 non-null  object
dtypes: int64(3), object(5)
memory usage: 41.7+ MB


In [88]:
# Turn hashtags into lists with all lowercase

df['hashtags'] = df['hashtags'].str.lower()
df['hashtags'] = df.apply(lambda row:  row['hashtags'].replace(' ', ',').split(','), axis=1)

In [89]:
# Create a list of all hashtags

hashtags_all = []
for row in df['hashtags']:
    for item in row:
        hashtags_all.append(item.lower())
        
# see hashtag count and sort by frequency
unique_hashtags = list(pd.Series(hashtags_all).value_counts().index)
# view top 5 for reference
print(unique_hashtags[:5])

['climatechange', 'climateaction', 'environment', 'energy', 'actonclimate']


In [90]:
# Believer Hashtags
believer_tags = ['climatechangeisreal', 'actonclimate', 'extinctionrebellion', 'climateemergency', 
                 'climateactionnow', 'capitalism', 'public_health', 'climateaction', 'humanityextinction',
                 'activism', 'noplanetb', 'savetheplanet', 'climateaction']

# Denier Hashtags
denier_tags = ['climatechangeisfalse', 'climatechangenotreal', 'climatechangehoax', 
               'globalwarminghoax', 'tcot', 'ccot', 'tlot', 'pjnet', 'rednationrising', 'votered', 
               'libtard', 'libtards', 'maga', 'climatedeniers', 'climatehoax', 'globalcooling',
              'climatechangescam', 'climatehysteria', 'globalwarmingisahoax', 'globalwarmingscam', 'globalcooling']

In [91]:
# assign believer status to each row in the dataset
believe_series = []

for idx, row in df['hashtags'].iteritems():
    # set a count for matching tags in row
    believe = 0
    deny = 0 
    for tag in row:
        if tag.lower() in denier_tags:
            deny += 1
        elif tag.lower() in believer_tags:
            believe += 1
    
    # check that only one type of tag appears
    if (believe > 0) and (deny == 0):
        believe_series.append(1)
    elif (believe == 0) and (deny > 0):
        believe_series.append(0)
    else: # <-- if no matching tag OR if both appear, label as unknown using nonetype 
        believe_series.append(None)

In [92]:
# create column in dataframe for the believer_series

df = df.assign(believer = believe_series)

# Remove NA's

df_believer_status = df.dropna(subset=['believer'])

# subset dataframe to only include positive and negative cases

df = df.dropna(subset = ['believer'])

df['believer'].value_counts()

1.0    459406
0.0     82438
Name: believer, dtype: int64

In [ ]:
# Turn believer column into target labels of 0 and 1

df['believer'] = df['believer'].astype(int)
df = df.rename(columns = {'believer':'target'})

In [94]:
df['target'].value_counts()

1    459406
0     82438
Name: target, dtype: int64

In [95]:
# Save file

df.to_pickle("training_dataset.pkl")